Figuring out Web Scraping!! Lets do this

In [74]:
import requests #imports
from bs4 import BeautifulSoup #https://realpython.com/beautiful-soup-web-scraper-python/
import pandas as pd

#this copies the HTML from the URL

URL = "https://www.espn.com/nfl/player/stats/_/id/2969962/duke-johnson"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [75]:
#Get only the container of the information, ignore the rest of the HTML
results = soup.find(id="fittPageContainer")

In [76]:
#Makes the HTML "Readable"
print(results.prettify())

<div id="fittPageContainer">
 <main aria-label="no-content" hidden="" id="main" tabindex="-1">
 </main>
 <div data-box-type="fitt-adbox-exclusions">
  <div class="Ad ad-slot" data-category-exclusion="true" data-slot-kvps="pos=exclusions" data-slot-type="exclusions">
  </div>
 </div>
 <div class="StickyContainer" data-sticky-parent="true">
  <div class="ResponsiveWrapper">
   <div class="PlayerHeader w-100 overflow-hidden bb PlayerHeader--desktop-lg bg-clr-white brdr-clr-gray-07">
    <div class="PlayerHeader__Container w-100 center flex relative items-center justify-between">
     <div class="PlayerHeader__Left flex items-center justify-start overflow-hidden brdr-clr-gray-09">
      <div class="PlayerHeader__Main flex items-center">
       <div class="PlayerHeader__Image_Container overflow-hidden bg-clr-gray-09">
        <div class="PlayerHeader__Image">
         <figure class="Image aspect-ratio--parent PlayerHeader__Logo">
          <div class="RatioFrame aspect-ratio--1x1">
        

In [77]:
#These are the elements of the player that don't ever change
immutable_elements = results.find_all("div", class_="fw-medium clr-black")

In [78]:
for element in immutable_elements:
    print(element, end="\n"*2)

<div class="fw-medium clr-black"><div>5' 9", 210 lbs</div></div>

<div class="fw-medium clr-black"><div>9/23/1993 (28)</div></div>

<div class="fw-medium clr-black"><div><a class="AnchorLink clr-black" data-clubhouse-uid="s:20~l:23~t:2390" href="https://www.espn.com/college-football/team/_/id/2390/miami-hurricanes" tabindex="0">Miami</a></div></div>

<div class="fw-medium clr-black"><div>2015: Rd 3, Pk 77 (CLE)</div></div>

<div class="fw-medium clr-black"><div><span class="TextStatus TextStatus--green plain ml0">Active</span></div></div>



In [79]:
#Get the elements from the html
print(immutable_elements[0].text)
print(immutable_elements[1].text)
print(immutable_elements[2].text)
print(immutable_elements[3].text)
print(immutable_elements[4].text)

5' 9", 210 lbs
9/23/1993 (28)
Miami
2015: Rd 3, Pk 77 (CLE)
Active


In [80]:
HTWT = immutable_elements[0].text.strip()
DOB = immutable_elements[1].text.strip()
COLLEGE = immutable_elements[2].text.strip()
Draftpick = immutable_elements[3].text.strip()
activity_status = immutable_elements[4].text.strip()

In [81]:
#Try to get the name

full_name = results.find_all("div", class_="PlayerHeader__Main_Aside min-w-0 flex-grow flex-basis-0")

In [82]:
for element in full_name:
    name = element.find("h1", class_="PlayerHeader__Name")
    print(name.text.strip())

DukeJohnson


In [83]:
#try the team and position
for element in full_name:
    pos = element.find("ul", class_="PlayerHeader__Team_Info list flex pt1 pr4 min-w-0 flex-basis-0 flex-shrink flex-grow nowrap")
    print(pos.text.strip())

Buffalo Bills#22Running Back


In [84]:
pos.text.strip()

'Buffalo Bills#22Running Back'

In [85]:
#isolate team name
team = pos.text.strip().split('#')[0] #https://www.w3schools.com/python/ref_string_split.asp

In [86]:
print(team)

Buffalo Bills


In [87]:
numrole = pos.text.strip().split('#')[1]

In [88]:
numrole

'22Running Back'

In [89]:
#isolate Role on team
role = ''.join(i for i in numrole if not i.isdigit()) #https://stackoverflow.com/questions/12851791/removing-numbers-from-string

In [90]:
role

'Running Back'

SCRAPE THE BIG TABLES

In [91]:
tables = results.find_all("div", class_="ResponsiveTable ResponsiveTable--fixed-left pt4")

In [115]:
#Need to make this automatic
passing = tables[0]
rushing = tables[1]
returning = tables[2]
defensive = tables[3]
scoring = tables[4]
kicking = tables[5]

In [116]:
passing1 = passing.find("table", class_="Table Table--align-right Table--fixed Table--fixed-left")

In [117]:
# Obtain every title of columns with tag <th>
headers = []
for i in passing1.find_all("th"):
 title = i.text
 headers.append(title)

In [118]:
headers

['season', 'Team']

In [119]:
# Create a dataframe
passing1data = pd.DataFrame(columns = headers)

In [120]:
# Create a for loop to fill mydata
for j in passing1.find_all("tr")[1:]:
 row_data = j.find_all("td")
 row = [i.text for i in row_data]
 length = len(passing1data)
 passing1data.loc[length] = row

In [121]:
passing1data

season Team
0    2015  CLE
1    2016  CLE
2    2017  CLE
3    2018  CLE
4    2019  HOU
5    2020  HOU
6    2021  MIA
7  Career

In [122]:
passing2 = passing.find("table", class_="Table Table--align-right")
passing2.text

'GPATTYDSAVGTDLNGFDFUMLST161043793.6039190016733584.9122171116823484.2419261016402015.002390016834104.9240171111772353.112311225713304.63271800965302,2614.3114011754'

In [123]:
# Obtain every title of columns with tag <th>
headers = []
for i in passing2.find_all("th"):
 title = i.text
 headers.append(title)
headers

['GP', 'ATT', 'YDS', 'AVG', 'TD', 'LNG', 'FD', 'FUM', 'LST']

In [124]:
# Create a dataframe
passing2data = pd.DataFrame(columns = headers)

# Create a for loop to fill mydata
for j in passing2.find_all("tr")[1:]:
 row_data = j.find_all("td")
 row = [i.text for i in row_data]
 length = len(passing2data)
 passing2data.loc[length] = row
    
passing2data

GP  ATT    YDS  AVG  TD LNG   FD FUM LST
0  16  104    379  3.6   0  39   19   0   0
1  16   73    358  4.9   1  22   17   1   1
2  16   82    348  4.2   4  19   26   1   0
3  16   40    201  5.0   0  23    9   0   0
4  16   83    410  4.9   2  40   17   1   1
5  11   77    235  3.1   1  23   11   2   2
6   5   71    330  4.6   3  27   18   0   0
7  96  530  2,261  4.3  11  40  117   5   4

In [125]:
#combine passing 1 and 2
passingdata = pd.concat([passing1data,passing2data], axis = 1) #https://www.geeksforgeeks.org/how-to-combine-two-dataframe-in-python-pandas/

#drop row 4
passingdata.drop(labels = [4], axis = 0, inplace = True)

display(passingdata)

season Team  GP  ATT    YDS  AVG  TD LNG   FD FUM LST
0    2015  CLE  16  104    379  3.6   0  39   19   0   0
1    2016  CLE  16   73    358  4.9   1  22   17   1   1
2    2017  CLE  16   82    348  4.2   4  19   26   1   0
3    2018  CLE  16   40    201  5.0   0  23    9   0   0
5    2020  HOU  11   77    235  3.1   1  23   11   2   2
6    2021  MIA   5   71    330  4.6   3  27   18   0   0
7  Career       96  530  2,261  4.3  11  40  117   5   4

Process a single table from scratch

In [126]:
#Find the first part of the table from the html
rushing1 = rushing.find("table", class_="Table Table--align-right Table--fixed Table--fixed-left")

# Obtain every title of columns with tag <th>
headers = []
for i in rushing1.find_all("th"):
 title = i.text
 headers.append(title)
    
# Create a dataframe
rushing1data = pd.DataFrame(columns = headers)

# Create a for loop to fill table
for j in rushing1.find_all("tr")[1:]:
 row_data = j.find_all("td")
 row = [i.text for i in row_data]
 length = len(rushing1data)
 rushing1data.loc[length] = row
    
rushing2 = rushing.find("table", class_="Table Table--align-right")

# Obtain every title of columns with tag <th>
headers = []
for i in rushing2.find_all("th"):
 title = i.text
 headers.append(title)

#Add Rushing tag to all headers
append_str = 'Rushing - '
headers = [append_str + sub for sub in headers] #https://www.geeksforgeeks.org/python-append-suffix-prefix-to-strings-in-list/
    
    
# Create a dataframe
rushing2data = pd.DataFrame(columns = headers)

# Create a for loop to fill mydata
for j in rushing2.find_all("tr")[1:]:
 row_data = j.find_all("td")
 row = [i.text for i in row_data]
 length = len(rushing2data)
 rushing2data.loc[length] = row

rushingdata = pd.concat([rushing1data,rushing2data], axis = 1)
rushingdata.drop(rushingdata.tail(1).index,inplace=True) #https://stackoverflow.com/questions/26921651/how-to-delete-the-last-row-of-data-of-a-pandas-dataframe
display(rushingdata)

season Team Rushing - GP Rushing - REC Rushing - TGTS Rushing - YDS  \
0   2015  CLE           16            61             74           534   
1   2016  CLE           16            53             72           514   
2   2017  CLE           16            74             93           693   
3   2018  CLE           16            47             62           429   
4   2019  HOU           16            44             62           410   
5   2020  HOU           11            28             35           249   
6   2021  MIA            5             4              5            41   

  Rushing - AVG Rushing - TD Rushing - LNG Rushing - FD Rushing - FUM  \
0           8.8            2            52           23             1   
1           9.7            0            32           24             1   
2           9.4            3            41           31             2   
3           9.1            3            32           25             1   
4           9.3            3            21           18             0   
5           8.9            1            48            8             1   
6          10.3            0            20            3             0   

  Rushing - LST  
0             0  
1             0  
2             2  
3             0  
4             0  
5             0  
6             0

In [127]:
#build template function for seperate stats, source: https://www.w3schools.com/python/python_functions.asp
def rushingtable(rushing):
    #Find the first part of the table from the html
    rushing1 = rushing.find("table", class_="Table Table--align-right Table--fixed Table--fixed-left")
    
    # Obtain every title of columns with tag <th>
    headers = []
    for i in rushing1.find_all("th"):
        title = i.text
        headers.append(title)
    
    # Create a dataframe
    rushing1data = pd.DataFrame(columns = headers)
    
    # Create a for loop to fill table
    for j in rushing1.find_all("tr")[1:]:
        row_data = j.find_all("td")
        row = [i.text for i in row_data]
        length = len(rushing1data)
        rushing1data.loc[length] = row
    
    #find the second part of the table from the html
    rushing2 = rushing.find("table", class_="Table Table--align-right")
    
    # Obtain every title of columns with tag <th>
    headers = []
    for i in rushing2.find_all("th"):
        title = i.text
        headers.append(title)
    
    #Add Rushing tag to all headers
    append_str = 'Rushing: '
    headers = [append_str + sub for sub in headers]
    
    # Create a dataframe source:https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/
    rushing2data = pd.DataFrame(columns = headers)
    
    # Create a for loop to fill mydata
    for j in rushing2.find_all("tr")[1:]:
        row_data = j.find_all("td")
        row = [i.text for i in row_data]
        length = len(rushing2data)
        rushing2data.loc[length] = row
        
    rushingdata = pd.concat([rushing1data,rushing2data], axis = 1)
    rushingdata.drop(rushingdata.tail(1).index,inplace=True) 
    display(rushingdata)

In [131]:
rushingtable(returning)

ValueError: cannot set a row with mismatched columns

In [129]:
#Make function generalized
def maketable(data, statname):
    #Find the first part of the table from the html
    data1 = data.find("table", class_="Table Table--align-right Table--fixed Table--fixed-left")
    
    # Obtain every title of columns with tag <th>
    headers = []
    for i in data1.find_all("th"):
        title = i.text
        headers.append(title)
    
    # Create a dataframe
    data1data = pd.DataFrame(columns = headers)
    
    # Create a for loop to fill table
    for j in data1.find_all("tr")[1:]:
        row_data = j.find_all("td")
        row = [i.text for i in row_data]
        length = len(data1data)
        data1data.loc[length] = row
    
    #find the second part of the table from the html
    data2 = data.find("table", class_="Table Table--align-right")
    
    # Obtain every title of columns with tag <th>
    headers = []
    for i in data2.find_all("th"):
        title = i.text
        headers.append(title)
    
    #Add name tag to all headers
    append_str = statname
    headers = [append_str + sub for sub in headers]
    
    # Create a dataframe
    data2data = pd.DataFrame(columns = headers)
    
    # Create a for loop to fill mydata
    for j in data2.find_all("tr")[1:]:
        row_data = j.find_all("td")
        row = [i.text for i in row_data]
        length = len(data2data)
        data2data.loc[length] = row
        
    datadata = pd.concat([data1data,data2data], axis = 1)
    datadata.drop(datadata.tail(1).index,inplace=True)
    return datadata

In [130]:
a = maketable(returning, "returning")
a

ValueError: cannot set a row with mismatched columns

In [109]:
substr = "Passing" 
if "Passing"  in passing.text:
    print(True)
else:
    print(False)

False


In [110]:
passing.text

'RushingseasonTeam2015CLE2016CLE2017CLE2018CLE2019HOU2020HOU2021MIACareerGPATTYDSAVGTDLNGFDFUMLST161043793.6039190016733584.9122171116823484.2419261016402015.002390016834104.9240171111772353.112311225713304.63271800965302,2614.3114011754'